# Modeling - Housing Data Ames, Iowa

Jeremy Herrmann

Import what we will use.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn.linear_model as lin
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.neighbors import KNeighborsRegressor
from statistics import variance as var
import sklearn.metrics
from sklearn.ensemble import RandomForestRegressor

Load the data and split it into a train and test set.

In [2]:
train_whole = pd.read_csv("train_clean_csv.csv", index_col=0)
#test = pd.read_csv("test_clean_csv.csv", index_col=0)
target_whole = train_whole['SalePrice']
train = train_whole.iloc[:1000,:]
test = train_whole.iloc[1000:,:]
train_target = train["SalePrice"]
test_target = test["SalePrice"]

Create variables that contain the names of different features so that I can access them more easily later on.

In [3]:
cont_feat = ["LotFrontage", "LotArea", "YearBuilt", "YearRemodAdd", "BsmtFinSF1", "MasVnrArea",
                   "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", 
                "LowQualFinSF", "GrLivArea", "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", "BedroomAbvGr", "KitchenAbvGr",
                   "TotRmsAbvGrd", "Fireplaces", "GarageCars", "GarageArea", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch",
                   "3SsnPorch", "ScreenPorch", "PoolArea", "MiscVal"]

In [4]:
ord_feat = ['Street', 'LotShape', 'Utilities', 'LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 
            'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual',
           'Functional', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC']

In [5]:
dum_feat = ['MSSubClass_SC160','MSSubClass_SC180','MSSubClass_SC190','MSSubClass_SC20','MSSubClass_SC30',
 'MSSubClass_SC40','MSSubClass_SC45','MSSubClass_SC50','MSSubClass_SC60','MSSubClass_SC70','MSSubClass_SC75',
 'MSSubClass_SC80','MSSubClass_SC85','MSSubClass_SC90','MSZoning_FV','MSZoning_RH','MSZoning_RL','MSZoning_RM',
 'Alley_2','Alley_None','LandContour_HLS','LandContour_Low','LandContour_Lvl','LotConfig_CulDSac','LotConfig_FR2','LotConfig_FR3',
 'LotConfig_Inside','Neighborhood_Blueste','Neighborhood_BrDale','Neighborhood_BrkSide','Neighborhood_ClearCr','Neighborhood_CollgCr',
 'Neighborhood_Crawfor','Neighborhood_Edwards','Neighborhood_Gilbert','Neighborhood_IDOTRR','Neighborhood_MeadowV',
 'Neighborhood_Mitchel','Neighborhood_NAmes','Neighborhood_NPkVill','Neighborhood_NWAmes','Neighborhood_NoRidge','Neighborhood_NridgHt',
 'Neighborhood_OldTown','Neighborhood_SWISU','Neighborhood_Sawyer','Neighborhood_SawyerW','Neighborhood_Somerst','Neighborhood_StoneBr',
 'Neighborhood_Timber','Neighborhood_Veenker','Condition1_Feedr','Condition1_Norm','Condition1_PosA','Condition1_PosN',
 'Condition1_RRAe','Condition1_RRAn','Condition1_RRNe','Condition1_RRNn','Condition2_Feedr','Condition2_Norm','Condition2_PosA',
 'Condition2_PosN','Condition2_RRAe','Condition2_RRAn','Condition2_RRNn','BldgType_2fmCon','BldgType_Duplex','BldgType_Twnhs','BldgType_TwnhsE',
 'HouseStyle_1.5Unf','HouseStyle_1Story','HouseStyle_2.5Fin','HouseStyle_2.5Unf','HouseStyle_2Story','HouseStyle_SFoyer',
 'HouseStyle_SLvl','RoofStyle_Gable','RoofStyle_Gambrel','RoofStyle_Hip','RoofStyle_Mansard','RoofStyle_Shed',
 'RoofMatl_CompShg','RoofMatl_Membran','RoofMatl_Metal','RoofMatl_Roll','RoofMatl_Tar&Grv','RoofMatl_WdShake',
 'RoofMatl_WdShngl','Exterior1st_AsphShn','Exterior1st_BrkComm',
 'Exterior1st_BrkFace',
 'Exterior1st_CBlock',
 'Exterior1st_CemntBd',
 'Exterior1st_HdBoard',
 'Exterior1st_ImStucc',
 'Exterior1st_MetalSd',
 'Exterior1st_Plywood',
 'Exterior1st_Stone',
 'Exterior1st_Stucco',
 'Exterior1st_VinylSd',
 'Exterior1st_Wd Sdng',
 'Exterior1st_WdShing',
 'Exterior2nd_AsphShn',
 'Exterior2nd_Brk Cmn',
 'Exterior2nd_BrkFace',
 'Exterior2nd_CBlock',
 'Exterior2nd_CmentBd',
 'Exterior2nd_HdBoard',
 'Exterior2nd_ImStucc',
 'Exterior2nd_MetalSd',
 'Exterior2nd_Other',
 'Exterior2nd_Plywood',
 'Exterior2nd_Stone',
 'Exterior2nd_Stucco',
 'Exterior2nd_VinylSd',
 'Exterior2nd_Wd Sdng',
 'Exterior2nd_Wd Shng',
 'MasVnrType_BrkFace',
 'MasVnrType_None',
 'MasVnrType_Stone',
 'Foundation_CBlock',
 'Foundation_PConc',
 'Foundation_Slab',
 'Foundation_Stone',
 'Foundation_Wood',
 'Heating_GasA',
 'Heating_GasW',
 'Heating_Grav',
 'Heating_OthW',
 'Heating_Wall',
 'CentralAir_Y',
 'Electrical_FuseF',
 'Electrical_FuseP',
 'Electrical_Mix',
 'Electrical_SBrkr',
 'Electrical_Sbrkr',
 'GarageType_Attchd',
 'GarageType_Basment',
 'GarageType_BuiltIn',
 'GarageType_CarPort',
 'GarageType_Detchd',
 'GarageType_No',
 'GarageFinish_No',
 'GarageFinish_RFn',
 'GarageFinish_Unf',
 'Fence_GdWo',
 'Fence_MnPrv',
 'Fence_MnWw',
 'Fence_No',
 'MiscFeature_No',
 'MiscFeature_Othr',
 'MiscFeature_Shed',
 'MiscFeature_TenC',
 'MoSold_Aug',
 'MoSold_Dec',
 'MoSold_Feb',
 'MoSold_Jan',
 'MoSold_Jul',
 'MoSold_Jun',
 'MoSold_Mar',
 'MoSold_May',
 'MoSold_Nov',
 'MoSold_Oct',
 'MoSold_Sep',
 'YrSold_2007',
 'YrSold_2008',
 'YrSold_2009',
 'YrSold_2010',
 'SaleType_CWD',
 'SaleType_Con',
 'SaleType_ConLD',
 'SaleType_ConLI',
 'SaleType_ConLw',
 'SaleType_New',
 'SaleType_Oth',
 'SaleType_WD',
 'SaleCondition_AdjLand',
 'SaleCondition_Alloca',
 'SaleCondition_Family',
 'SaleCondition_Normal',
 'SaleCondition_Partial']

In [6]:
num_feat = cont_feat + ord_feat

In [23]:
num_sale_feat = num_feat + ['SalePrice']

In [7]:
top_feat = ['GrLivArea', 'OverallQual', 'YearBuilt']

Define a function to calculate the Root Mean Squared Error which it the scoring metric for the Kaggle competiton

In [8]:
def rmse(resid):
    return np.sqrt(resid ** 2).mean()

### Linear

I will train a linear model using the top 3 features and compare that with a linear model that uses all of the features.

In [9]:
L_1 = lin.LinearRegression()
L_1.fit(train[top_feat],train_target)
print( L_1.score(test[top_feat],test_target) )
L_1_predictions = L_1.predict(test[top_feat])
L_1_resid = test_target - L_1_predictions
print( rmse(L_1_resid) )
L_1_scores = -cross_val_score(L_1, train_whole[top_feat], target_whole, cv=5, scoring='mean_squared_error')
print( np.mean(np.sqrt(L_1_scores)) )
print( np.sqrt(L_1_scores) )

0.761274909055
0.13998575586543807
0.182494731186
[ 0.17576618  0.18896165  0.18399367  0.16876171  0.19499045]


The linear model train using the top 3 features explains 76% of the variation in the test set and has an RMSE of about .14.  For comparison, the best Kaggle score RMSE is .10 and halfway down the leaderboard is .13.  When using crossvalscore on all of the data, the average RMSE was .18.

In [10]:
L_2 = lin.LinearRegression()
L_2.fit(train[cont_feat + ord_feat + dum_feat],train_target)
print( L_2.score(test[cont_feat + ord_feat + dum_feat],test_target) )
L_2_predictions = L_2.predict(test[cont_feat + ord_feat + dum_feat])
L_2_resid = test_target - L_2_predictions
print( rmse(L_2_resid) )
L_2_scores = -cross_val_score(L_2, train_whole[cont_feat + ord_feat + dum_feat], target_whole, cv=5, scoring='mean_squared_error')
print( np.sqrt(L_2_scores) )

-1.32078218828e+22
5966363424.362151
[  2.36765182e-01   1.28232205e+11   8.08825059e+10   2.68559473e+11
   3.78963440e+10]


I get crazy things that make no sense when I use the dummy features.  I suspect that maybe there is redundancy amongst dummy columns of different variables causing the linear algebra not to work?

In [11]:
L_3 = lin.LinearRegression()
L_3.fit(train[cont_feat + ord_feat],train_target)
print( L_3.score(test[cont_feat + ord_feat],test_target) )
L_3_predictions = L_3.predict(test[cont_feat + ord_feat])
L_3_resid = test_target - L_3_predictions
print( rmse(L_3_resid) )
L_3_scores = -cross_val_score(L_3, train_whole[cont_feat + ord_feat], target_whole, cv=5, scoring='mean_squared_error')
print( np.mean(np.sqrt(L_3_scores)) )
print( np.sqrt(L_3_scores) )

0.843823264681
0.09231925999627534
0.252386354196
[ 0.1197511   0.15278841  0.13391187  0.68416393  0.17131647]


The model seems to do worse with all continuous and ordinal features.  There is something about that 4th test.

### Nearest Neighbor

In [12]:
K_1 = KNeighborsRegressor(n_neighbors=25, weights="distance")
K_1.fit(train[top_feat],train_target)
print( K_1.score(test[top_feat],test_target) )
K_1_predictions = K_1.predict(test[top_feat])
K_1_resid = test_target - K_1_predictions
print( rmse(K_1_resid) )
K_1_scores = -cross_val_score(K_1, train_whole[top_feat], target_whole, cv=5, scoring='mean_squared_error')
print( np.mean(np.sqrt(K_1_scores)) )
print( np.sqrt(K_1_scores) )

0.796768666574
0.13057056080129611
0.17737205328
[ 0.17653408  0.18612877  0.18297703  0.16400007  0.17722031]


Not suprisingly, this performs similarly to the linear model trained on the same features.

### Random Forest

Building a random forest with the top features as well, produces similar results.

In [13]:
R_1 = RandomForestRegressor(n_estimators = 50)
R_1.fit(train[top_feat],train_target)
print( R_1.score(test[top_feat],test_target) )
R_1_predictions = R_1.predict(test[top_feat])
R_1_resid = test_target - R_1_predictions
print( rmse(R_1_resid) )
R_1_scores = -cross_val_score(R_1, train_whole[top_feat], target_whole, cv=5, scoring='mean_squared_error')
print( np.mean(np.sqrt(R_1_scores)) )
print( np.sqrt(R_1_scores) )

0.776132901726
0.13735546986522956
0.18566722736
[ 0.18810573  0.19864704  0.18969268  0.168054    0.1838367 ]


In [14]:
R_2 = RandomForestRegressor(n_estimators = 500)
R_2.fit(train[cont_feat + ord_feat + dum_feat],train_target)
print( R_2.score(test[cont_feat + ord_feat + dum_feat],test_target) )
R_2_predictions = R_2.predict(test[cont_feat + ord_feat + dum_feat])
R_2_resid = test_target - R_2_predictions
print( rmse(R_2_resid) )
R_2_scores = -cross_val_score(R_2, train_whole[cont_feat + ord_feat + dum_feat], target_whole, cv=5, scoring='mean_squared_error')
print( np.mean(np.sqrt(R_2_scores)) )
print( np.sqrt(R_2_scores) )

0.863643155403
0.09970326056792062
0.142311740386
[ 0.13633334  0.15296239  0.14325683  0.12968726  0.14931888]


A random forest with all features performs well.

In [15]:
features = cont_feat + ord_feat + dum_feat

feature_importances = R_2.feature_importances_

features_df = pd.DataFrame({'Features': features, 'Importance Score': feature_importances})
features_df.sort('Importance Score', inplace=True, ascending=False)

features_df.head(), features_df.tail()

C:\Users\jherrmann\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


(       Features  Importance Score
 34  OverallQual          0.551454
 12    GrLivArea          0.111915
 8   TotalBsmtSF          0.045081
 21   GarageCars          0.029651
 9      1stFlrSF          0.026245,                 Features  Importance Score
 140  Exterior1st_AsphShn               0.0
 204     MiscFeature_TenC               0.0
 136        RoofMatl_Roll               0.0
 180         Heating_OthW               0.0
 187     Electrical_Sbrkr               0.0)

### Building One Step at a Time

I will add the most highly correlated feature one at a time, and then find the next feature by looking at what is correlated with the residuals.  Using only linear regression, I end up with a cross val score average of .15.  Similar results come from a random forest using the same features.  I end up using ['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1', 'OverallCond'].  I want to try more ways in which I can also incorporate nearest neighbor and some other features if they are useful.  I will also try to install xgboost and maybe I will ask you for help next week if I am unsuccessful.  I feel like I would need xgboost to get a score higher than average on the Kaggle leaderboard>

In [18]:
corr_with_resid = train[num_feat + ["SalePrice"]].corr()
print( corr_with_resid['SalePrice'].sort_values(ascending=False).head() )
corr_with_resid['SalePrice'].sort_values(ascending=False).tail()

SalePrice      1.000000
OverallQual    0.821378
GrLivArea      0.748481
GarageCars     0.690545
ExterQual      0.682985
Name: SalePrice, dtype: float64


EnclosedPorch   -0.131662
KitchenAbvGr    -0.139685
LotShape        -0.263545
YearRemodAdd    -0.588875
YearBuilt       -0.590868
Name: SalePrice, dtype: float64

In [21]:
L_4 = lin.LinearRegression()
L_4.fit(train[['OverallQual']],train_target)
print( L_4.score(test[['OverallQual']],test_target) )
L_4_predictions = L_4.predict(test[['OverallQual']])
L_4_resid = test_target - L_4_predictions
print( rmse(L_4_resid) )
L_4_scores = -cross_val_score(L_4, train_whole[['OverallQual']], target_whole, cv=5, scoring='mean_squared_error')
print( np.mean(np.sqrt(L_4_scores)) )
print( np.sqrt(L_4_scores) )

L_4_predictions_t = L_4.predict(train[['OverallQual']])
L_4_resid_t = train_target - L_4_predictions_t

0.651542234375
0.17262514643998877
0.229726020832
[ 0.22320136  0.24919432  0.23368375  0.21268233  0.22986834]


In [36]:
corr_with_resid = train[num_feat].join(L_4_resid_t).corr()
print( corr_with_resid['SalePrice'].sort_values(ascending=False).head() )
corr_with_resid['SalePrice'].sort_values(ascending=False).tail()

SalePrice     1.000000
LotArea       0.445630
1stFlrSF      0.427286
GrLivArea     0.409588
BsmtFinSF1    0.355845
Name: SalePrice, dtype: float64


EnclosedPorch   -0.079759
LandSlope       -0.171876
YearBuilt       -0.216555
YearRemodAdd    -0.216702
LotShape        -0.218905
Name: SalePrice, dtype: float64

In [37]:
L_5 = lin.LinearRegression()
L_5.fit(train[['OverallQual','LotArea']],train_target)
print( L_5.score(test[['OverallQual','LotArea']],test_target) )
L_5_predictions = L_5.predict(test[['OverallQual','LotArea']])
L_5_resid = test_target - L_5_predictions
print( rmse(L_5_resid) )
L_5_scores = -cross_val_score(L_5, train_whole[['OverallQual','LotArea']], target_whole, cv=5, scoring='mean_squared_error')
print( np.mean(np.sqrt(L_5_scores)) )
print( np.sqrt(L_5_scores) )

L_5_predictions_t = L_5.predict(train[['OverallQual','LotArea']])
L_5_resid_t = train_target - L_5_predictions_t

0.716408873423
0.15178862844459268
0.205757278886
[ 0.19692047  0.22119704  0.20825042  0.19713576  0.2052827 ]


In [38]:
corr_with_resid = train[num_feat].join(L_5_resid_t).corr()
print( corr_with_resid['SalePrice'].sort_values(ascending=False).head() )
corr_with_resid['SalePrice'].sort_values(ascending=False).tail()

SalePrice       1.000000
GrLivArea       0.325252
BsmtFinSF1      0.299122
BsmtFinType1    0.292778
1stFlrSF        0.278444
Name: SalePrice, dtype: float64


LotShape        -0.079575
BsmtUnfSF       -0.089788
EnclosedPorch   -0.117329
YearRemodAdd    -0.279429
YearBuilt       -0.285864
Name: SalePrice, dtype: float64

In [39]:
L_6 = lin.LinearRegression()
L_6.fit(train[['OverallQual','LotArea', 'GrLivArea']],train_target)
print( L_6.score(test[['OverallQual','LotArea', 'GrLivArea']],test_target) )
L_6_predictions = L_6.predict(test[['OverallQual','LotArea', 'GrLivArea']])
L_6_resid = test_target - L_6_predictions
print( rmse(L_6_resid) )
L_6_scores = -cross_val_score(L_6, train_whole[['OverallQual','LotArea', 'GrLivArea']], target_whole, cv=5, scoring='mean_squared_error')
print( np.mean(np.sqrt(L_6_scores)) )
print( np.sqrt(L_6_scores) )

L_6_predictions_t = L_6.predict(train[['OverallQual','LotArea', 'GrLivArea']])
L_6_resid_t = train_target - L_6_predictions_t

0.742849888173
0.14230580888179942
0.188169630136
[ 0.17905187  0.19682267  0.18839104  0.17789251  0.19869005]


In [41]:
corr_with_resid = train[num_feat].join(L_6_resid_t).corr()
print( corr_with_resid['SalePrice'].sort_values(ascending=False).head() )
corr_with_resid['SalePrice'].sort_values(ascending=False).tail()

SalePrice       1.000000
BsmtFinType1    0.386739
BsmtFinSF1      0.356313
PavedDrive      0.312140
BsmtFullBath    0.310068
Name: SalePrice, dtype: float64


BsmtUnfSF       -0.142603
2ndFlrSF        -0.153694
EnclosedPorch   -0.184973
YearRemodAdd    -0.304734
YearBuilt       -0.366433
Name: SalePrice, dtype: float64

In [42]:
L_7 = lin.LinearRegression()
L_7.fit(train[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt']],train_target)
print( L_7.score(test[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt']],test_target) )
L_7_predictions = L_7.predict(test[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt']])
L_7_resid = test_target - L_7_predictions
print( rmse(L_7_resid) )
L_7_scores = -cross_val_score(L_7, train_whole[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt']], target_whole, cv=5, scoring='mean_squared_error')
print( np.mean(np.sqrt(L_7_scores)) )
print( np.sqrt(L_7_scores) )

L_7_predictions_t = L_7.predict(train[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt']])
L_7_resid_t = train_target - L_7_predictions_t

0.787415347728
0.12749665754089193
0.168713597849
[ 0.15578175  0.17493623  0.17117563  0.15577275  0.18590163]


In [43]:
corr_with_resid = train[num_feat].join(L_7_resid_t).corr()
print( corr_with_resid['SalePrice'].sort_values(ascending=False).head() )
corr_with_resid['SalePrice'].sort_values(ascending=False).tail()

SalePrice       1.000000
BsmtFinSF1      0.317679
OverallCond     0.311217
BsmtFinType1    0.292395
BsmtFullBath    0.279114
Name: SalePrice, dtype: float64


2ndFlrSF       -0.125568
KitchenAbvGr   -0.126940
BedroomAbvGr   -0.127234
BsmtUnfSF      -0.146730
YearRemodAdd   -0.188414
Name: SalePrice, dtype: float64

In [44]:
L_8 = lin.LinearRegression()
L_8.fit(train[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1']],train_target)
print( L_8.score(test[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1']],test_target) )
L_8_predictions = L_8.predict(test[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1']])
L_8_resid = test_target - L_8_predictions
print( rmse(L_8_resid) )
L_8_scores = -cross_val_score(L_8, train_whole[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1']], target_whole, cv=5, scoring='mean_squared_error')
print( np.mean(np.sqrt(L_8_scores)) )
print( np.sqrt(L_8_scores) )

L_8_predictions_t = L_8.predict(train[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1']])
L_8_resid_t = train_target - L_8_predictions_t

0.782094611645
0.11692238253333732
0.163379096442
[ 0.14932193  0.1684544   0.16114549  0.14335832  0.19461534]


In [45]:
corr_with_resid = train[num_feat].join(L_8_resid_t).corr()
print( corr_with_resid['SalePrice'].sort_values(ascending=False).head() )
corr_with_resid['SalePrice'].sort_values(ascending=False).tail()

SalePrice      1.000000
OverallCond    0.314205
HeatingQC      0.173959
KitchenQual    0.170612
Functional     0.170082
Name: SalePrice, dtype: float64


PoolQC         -0.049811
2ndFlrSF       -0.064459
BedroomAbvGr   -0.080745
KitchenAbvGr   -0.135345
YearRemodAdd   -0.210803
Name: SalePrice, dtype: float64

In [46]:
L_9 = lin.LinearRegression()
L_9.fit(train[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1', 'OverallCond']],train_target)
print( L_9.score(test[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1', 'OverallCond']],test_target) )
L_9_predictions = L_9.predict(test[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1', 'OverallCond']])
L_9_resid = test_target - L_9_predictions
print( rmse(L_9_resid) )
L_9_scores = -cross_val_score(L_9, train_whole[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1', 'OverallCond']], target_whole, cv=5, scoring='mean_squared_error')
print( np.mean(np.sqrt(L_9_scores)) )
print( np.sqrt(L_9_scores) )

L_9_predictions_t = L_9.predict(train[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1', 'OverallCond']])
L_9_resid_t = train_target - L_9_predictions_t

0.809585127306
0.10645309420047172
0.152450937011
[ 0.13810609  0.15648843  0.15271099  0.13050976  0.18443941]


In [47]:
corr_with_resid = train[num_feat].join(L_9_resid_t).corr()
print( corr_with_resid['SalePrice'].sort_values(ascending=False).head() )
corr_with_resid['SalePrice'].sort_values(ascending=False).tail()

SalePrice      1.000000
TotalBsmtSF    0.155863
1stFlrSF       0.148654
GarageArea     0.144222
KitchenQual    0.139939
Name: SalePrice, dtype: float64


MiscVal        -0.042146
KitchenAbvGr   -0.099614
2ndFlrSF       -0.101398
BedroomAbvGr   -0.110822
YearRemodAdd   -0.113576
Name: SalePrice, dtype: float64

In [53]:
#for i in ['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1', 'OverallCond']:

#    num_feat_minus = num_feat.remove(i)


print( num_feat )

['LotFrontage', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'Street', 'LotShape', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC']


In [60]:
R_3 = RandomForestRegressor(n_estimators = 50)
R_3.fit(train[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1', 'OverallCond']],train_target)
print( R_3.score(test[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1', 'OverallCond']],test_target) )
R_3_predictions = R_3.predict(test[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1', 'OverallCond']])
R_3_resid = test_target - R_3_predictions
print( rmse(R_3_resid) )
R_3_scores = -cross_val_score(R_3, train_whole[['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1', 'OverallCond']], target_whole, cv=5, scoring='mean_squared_error')
print( np.mean(np.sqrt(R_3_scores)) )
print( np.sqrt(R_3_scores) )

0.841016230965
0.11074243666404539
0.151579708698
[ 0.15141182  0.16004882  0.15124813  0.13860747  0.1565823 ]


In [61]:
features = ['OverallQual','LotArea', 'GrLivArea', 'YearBuilt', 'BsmtFinSF1', 'OverallCond']

feature_importances = R_3.feature_importances_

features_df = pd.DataFrame({'Features': features, 'Importance Score': feature_importances})
features_df.sort('Importance Score', inplace=True, ascending=False)

features_df.head(15), features_df.tail()

C:\Users\jherrmann\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


(      Features  Importance Score
 0  OverallQual          0.595659
 2    GrLivArea          0.179932
 3    YearBuilt          0.082980
 4   BsmtFinSF1          0.060282
 1      LotArea          0.054209
 5  OverallCond          0.026938,       Features  Importance Score
 2    GrLivArea          0.179932
 3    YearBuilt          0.082980
 4   BsmtFinSF1          0.060282
 1      LotArea          0.054209
 5  OverallCond          0.026938)